# SAS 해커톤에 쓰일 dummy data maker

In [186]:
import pandas as pd
import numpy as np
from faker import Faker
import random
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
from collections import defaultdict

In [3]:
fake_ko = Faker('ko_KR')

In [126]:
# 데이터는 총 num_samples 개
num_samples = 100000

In [173]:
# l번호 생성
def generate_longtermcare_id():
    prefix = 'L'
    year = str(random.randint(8, 24)).zfill(2)  # 2자리로 만들기 위해 zfill 사용
    middle = '00'
    random_number = str(random.randint(0, 99999)).zfill(5)  # 6자리로 만들기 위해 zfill 사용
    
    # 최종 인정관리번호 생성
    longtermcare_id = f"{prefix}{year}{middle}{random_number}"    
    return longtermcare_id

In [184]:
# l번호 생성 후 저장
print(f'sample: {generate_longtermcare_id()}')
longtermcare_id = np.array([generate_longtermcare_id() for _ in range(num_samples)])
longtermcare_id.shape, len(np.unique(longtermcare_id))

sample: L180014027


((100000,), 97066)

In [187]:
# l번호의 중복 제거
# 중복된 ID 찾기
unique_ids, counts = np.unique(longtermcare_id, return_counts=True)
duplicates = unique_ids[counts > 1]

# ID별로 인덱스 저장
id_to_indices = defaultdict(list)
for idx, id_value in enumerate(longtermcare_id):
    id_to_indices[id_value].append(idx)

# 중복된 ID 처리
for dup_id in duplicates:
    indices = id_to_indices[dup_id]
    # 첫 번째 ID는 그대로 두고, 나머지 중복된 ID 수정
    for count, idx in enumerate(indices[1:], start=1):
        new_middle = str(count).zfill(2)  # '01', '02', '03' 등으로 변경
        new_id = dup_id[:3] + new_middle + dup_id[5:]
        # 새로운 ID가 기존 ID와 중복되지 않도록 확인
        while new_id in longtermcare_id:
            count += 1
            new_middle = str(count).zfill(2)
            new_id = dup_id[:3] + new_middle + dup_id[5:]
        # 수정된 ID를 배열에 업데이트
        longtermcare_id[idx] = new_id
        # ID 인덱스 매핑 업데이트
        id_to_indices[new_id] = [idx]

# 최종 고유 ID의 수 출력
print(f"최종 고유 ID 수: {len(np.unique(longtermcare_id))}")

최종 고유 ID 수: 100000


In [118]:
# 초성, 중성, 종성 인덱스 리스트
#초성 = ㄱ  ㄴ ㄷ ㄹ ㅁ ㅂ  ㅅ  ㅇ ㅈ  ㅊ  ㅌ ㅍ ㅎ
#중성 = ㅏ ㅐ ㅒ ㅓ  ㅕ  ㅗ ㅘ   ㅛ ㅜ    ㅠ ㅡ ㅢ ㅣ
#종성 = ㄱ  ㄴ  ㄷ ㄹ   ㅁ ㅂ  ㅅ  ㅇ ㅈ 
choseong_list = [0, 2, 3, 5, 6, 7, 9, 11, 12, 14, 16, 17, 18]
jungseong_list = [0, 1, 3, 4, 6, 8, 9, 12, 13, 17, 18, 19, 20]
jongseong_list = [0, 1, 4, 7, 8, 16, 17, 19, 21, 22]  # 받침 없는 경우 0 포함
jongseong_weights = [5 if jongseong == 0 else 1 for jongseong in jongseong_list]

# 유니코드 한글 범위에서 종성을 분리하여 쌍자음 받침을 제외하는 함수
def generate_random_korean_char():

    choseong = random.choice(choseong_list)
    jungseong = random.choice(jungseong_list)
    jongseong = random.choices(jongseong_list, weights=jongseong_weights, k=1)[0]
        # 유니코드 한글 음절 계산
    syllable_code = 0xAC00 + (choseong * 21 * 28) + (jungseong * 28) + jongseong
    syllable_char = chr(syllable_code)
    return syllable_char
        
#랜덤이름 만들기
def generate_random_korean_name():
    # 성만들기
    last_name = fake_ko.last_name()

    # 유니코드 한글 범위 (가~힣)
    first_char = generate_random_korean_char()  # 초성부터 종성까지 한 글자
    second_char = generate_random_korean_char()  # 두 번째 글자
    # 성과 이름 결합
    full_name = last_name + first_char + second_char

    return full_name

In [119]:
# 이름 생성
print(f'sample: {generate_random_korean_name()}')
name = np.array([generate_random_korean_name() for _ in range(num_samples)])
name.shape, len(np.unique(name))

sample: 김팜순


((300000,), 294808)

In [136]:
# 생년월일
def generate_birthdates(min_age, max_age, std_dev, reference_date=datetime(2024, 8, 31)):
    """
    지정된 나이 범위와 표준편차를 기반으로 정규분포된 나이를 생성하고,
    해당 나이에 맞는 생년월일 리스트를 반환하는 함수입니다.

    인자:
    - min_age (int): 최소 나이
    - max_age (int): 최대 나이
    - std_dev (float): 나이의 표준편차
    - reference_date (datetime): 기준 날짜 (기본값: 2024년 8월 31일)

    반환값:
    - birthdates (list): 생년월일 문자열의 리스트 (형식: 'YYYY-MM-DD')
    """
    # 1. 생년월일 범위 설정
    earliest_birthdate = reference_date - relativedelta(years=max_age)
    latest_birthdate = reference_date - relativedelta(years=min_age)
    
    # 2. 전체 기간의 일 수 계산
    total_days = (latest_birthdate - earliest_birthdate).days
    
    # 3. 평균 나이 및 표준편차를 일 수로 변환
    mean_age = int((min_age + max_age) / 2)  # 정수형으로 변환
    mean_birthdate = reference_date - relativedelta(years=mean_age)
    mean_day_offset = (mean_birthdate - earliest_birthdate).days
    std_dev_days = std_dev * 365.25  # 표준편차를 일 수로 변환

    # 4. 정규분포를 따르는 일 수 생성
    day_offsets = np.random.normal(loc=mean_day_offset, scale=std_dev_days, size=1)
    day_offsets = np.clip(day_offsets, 0, total_days).astype(int)
    
    # 5. 생년월일 계산
    birthdates = [earliest_birthdate + timedelta(days=int(offset)) for offset in day_offsets]
    birthdates_str = [date.strftime('%Y-%m-%d') for date in birthdates]
    
    return birthdates_str


In [220]:
# 생년월일 생성
print(f'sample: {generate_birthdates(65, 100, 6, reference_date=datetime(2024, 8, 31))}')
birth_day = np.array([generate_birthdates(65, 100, 6, reference_date=datetime(2024, 8, 31)) for _ in range(num_samples)])
birth_day.shape, len(np.unique(birth_day))

sample: ['1944-09-16']


((100000, 1), 11084)

In [ ]:
address = pd.read_csv('./gps_data/merged_output.csv')
address

In [197]:
address.SIDO_NM.unique()

array(['강원특별자치도', '경기도', '경상남도', '경상북도', '광주광역시', '대구광역시', '대전광역시',
       '부산광역시', '서울특별시', '세종특별자치시', '울산광역시', '인천광역시', '전라남도', '전북특별자치도',
       '제주특별자치도', '충청남도', '충청북도'], dtype=object)

In [219]:
# 지역별 인구 비율 딕셔너리
population_distribution = {
    '서울특별시': 18.6,
    '경기도': 24.4,
    '부산광역시': 6.5,
    '인천광역시': 5.2,
    '대구광역시': 4.7,
    '광주광역시': 2.2,
    '대전광역시': 3.0,
    '울산광역시': 2.0,
    '세종특별자치시': 0.7,
    '강원특별자치도': 3.0,
    '충청북도': 3.2,
    '충청남도': 4.4,
    '전북특별자치도': 3.5,
    '전라남도': 3.3,
    '경상북도': 5.0,
    '경상남도': 7.4,
    '제주특별자치도': 1.7
}
# 지역 비율을 반영한 지역 선택 함수
def select_region_by_population():
    regions = list(population_distribution.keys())  # 지역 리스트
    weights = list(population_distribution.values())  # 각 지역에 대한 비율 리스트

    # random.choices를 사용하여 비율에 따라 지역을 선택
    selected_region = random.choices(regions, weights=weights, k=1)[0]
    
    return selected_region

In [221]:
# 소재지 생성
print(f'sample: {select_region_by_population()}')
region = np.array([select_region_by_population() for _ in range(num_samples)])
region.shape, len(np.unique(region))

sample: 충청남도


((100000,), 17)

In [236]:
unique_elements, counts = np.unique(region, return_counts=True)
sorted_indices = np.argsort(counts)[::-1]
# 결과 출력
for idx in sorted_indices:
    print(f'{unique_elements[idx]}: {counts[idx]}개')
print(f'{region.shape.values}')


경기도: 24682개
서울특별시: 18464개
경상남도: 7462개
부산광역시: 6532개
인천광역시: 5301개
경상북도: 5130개
대구광역시: 4866개
충청남도: 4570개
전북특별자치도: 3642개
전라남도: 3288개
충청북도: 3267개
강원특별자치도: 3069개
대전광역시: 3035개
광주광역시: 2211개
울산광역시: 1988개
제주특별자치도: 1736개
세종특별자치시: 757개


AttributeError: 'tuple' object has no attribute 'values'